In [3]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [4]:
import nltk
from nltk.tokenize import word_tokenize

# Check if the resource exists
nltk.data.find('tokenizers/punkt')
print("Punkt tokenizer is installed correctly.")


Punkt tokenizer is installed correctly.


In [1]:
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")


Python executable: c:\Users\user\Desktop\10 Acedamy W5\TELEGRAM_SCRAPPER\venv\Scripts\python.exe
Python version: 3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]


In [6]:
import asyncio
import logging
from telethon import TelegramClient
import csv
import os

# Set up logging
logging.basicConfig(level=logging.INFO, filename="scraper.log", filemode="w",
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Configurations
DATA_FOLDER = r"C:\Users\user\Desktop\10 Acedamy W5\All Data"
CHANNELS_FILE = os.path.join(DATA_FOLDER, "channels_to_crawl.csv")
SCRAPED_FILE = os.path.join(DATA_FOLDER, "scraped_data.csv")
API_ID = 20173022  # Replace with your API ID
API_HASH = 'bab4a3351ed7634a8c1a3f8767fcf75c'  # Replace with your API Hash


def load_channels(file_path):
    """Load channel usernames from a CSV file."""
    try:
        with open(file_path, 'r') as f:
            # Ensure each line is a list of strings, then we can access the correct column (1 index) and strip
            return [line[1].strip() for line in csv.reader(f) if len(line) > 1]
    except FileNotFoundError:
        logging.error(f"Channels file not found at {file_path}")
        return []


async def scrape_channel(client, channel, writer):
    """Scrape messages from a Telegram channel."""
    try:
        async for message in client.iter_messages(channel, limit=100):
            writer.writerow([channel, message.sender_id, message.date, message.text])
    except Exception as e:
        logging.error(f"Error scraping channel {channel}: {e}")


async def scrape_telegram_data():
    """Main function to scrape Telegram data."""
    logging.info("Starting Telegram scraper...")
    channels = load_channels(CHANNELS_FILE)
    if not channels:
        logging.error("No channels to scrape.")
        return

    async with TelegramClient('scraper', API_ID, API_HASH) as client:
        with open(SCRAPED_FILE, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(['channel', 'sender', 'timestamp', 'message'])
            for channel in channels:
                logging.info(f"Scraping channel: {channel}")
                await scrape_channel(client, channel, writer)

    logging.info("Scraping completed.")


# Use await for environments with an active event loop
if __name__ == "__main__":
    await scrape_telegram_data()


In [ ]:
task 2

In [9]:
import pandas as pd

# Define the entity types for labeling
ENTITY_TYPES = {
    "B-Product": "Product entity",
    "I-Product": "Product entity continuation",
    "B-LOC": "Location entity",
    "I-LOC": "Location entity continuation",
    "B-PRICE": "Price entity",
    "I-PRICE": "Price entity continuation",
    "O": "Other (no entity)"
}

# Example dataset (replace this with your dataset)
data = [
    ("Baby bottle እንግዲኛ አሁን ዋጋ 1000 ብር", "B-Product I-Product O B-PRICE I-PRICE O"),
    ("Addis Abeba በፍቅር እንቆቅልሽ", "B-LOC I-LOC O B-Product I-Product")
]

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data, columns=["Message", "Label"])

# Function to save labeled data in CoNLL format
def save_conll_format(df, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            message = row['Message']
            labels = row['Label'].split()
            message_tokens = message.split()

            # Iterate over tokens and corresponding labels
            for token, label in zip(message_tokens, labels):
                f.write(f"{token} {label}\n")
            
            f.write("\n")  # Blank line between sentences/messages

    print(f"Labeled data saved in CoNLL format to {file_path}")

# Save the labeled data to CoNLL format
save_conll_format(df, "labeled_data.conll")


Labeled data saved in CoNLL format to labeled_data.conll
